In [1]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

Node{T}

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [91]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

search (generic function with 1 method)

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [2]:
include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

In [3]:
using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end

genstensor (generic function with 2 methods)

In [4]:
l = 75
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,12);

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  0.368854 seconds (320.15 k allocations: 589.424 MB, 32.72% gc time)


In [6]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.107971 seconds (259.60 k allocations: 13.942 MB)
  0.080173 seconds (322.12 k allocations: 17.625 MB, 4.54% gc time)
  0.013778 seconds (7 allocations: 47.684 MB, 19.70% gc time)
  0.012109 seconds (7 allocations: 47.684 MB, 10.60% gc time)


In [7]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.338190 seconds (1.23 M allocations: 148.155 MB, 15.07% gc time)
  0.297563 seconds (1.21 M allocations: 147.300 MB, 15.78% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [8]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.046923 seconds (217.14 k allocations: 12.077 MB, 5.30% gc time)
  0.041683 seconds (211.89 k allocations: 11.764 MB)
  0.046537 seconds (217.14 k allocations: 12.077 MB, 5.47% gc time)
  0.018239 seconds (13 allocations: 47.684 MB, 7.72% gc time)
  0.015898 seconds (7 allocations: 47.684 MB, 4.35% gc time)
  0.049982 seconds (13 allocations: 47.684 MB, 1.66% gc time)


In [8]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

  3.900043 seconds (5.54 M allocations: 6.035 GB, 6.03% gc time)
  1.293228 seconds (176 allocations: 946.063 MB, 2.76% gc time)


In [197]:
macroexpand(:(@nloops 3 i x -> (x==3)? (1:size(T, x)): (1:i_{x+1}) begin
    s+= @nref 3 A x -> 3 - i
#    s += A[(@ntuple 3 x -> i_{4-x})...]
        end
))

quote  # cartesian.jl, line 31:
    for i_3 = 1:size(T,3) # cartesian.jl, line 32:
        nothing # cartesian.jl, line 33:
        begin  # cartesian.jl, line 31:
            for i_2 = 1:i_3 # cartesian.jl, line 32:
                nothing # cartesian.jl, line 33:
                begin  # cartesian.jl, line 31:
                    for i_1 = 1:i_2 # cartesian.jl, line 32:
                        nothing # cartesian.jl, line 33:
                        begin  # In[197], line 2:
                            s += A[3 - i,3 - i,3 - i]
                        end # cartesian.jl, line 34:
                        nothing
                    end
                end # cartesian.jl, line 34:
                nothing
            end
        end # cartesian.jl, line 34:
        nothing
    end
end

In [8]:
T1 = randn(5,5);
r = randn(10,10)
c = covbs(r);
c1 = cov(r, corrected = false);

In [3]:
function bstoarray{T <: AbstractFloat}(bsfunction::Function, dims::Int, bsdata::BoxStructure{T}...)
    (size(bsdata, 1) > 1)? testsize(bsdata...): ()
    s = size(bsdata[1])
    ret = zeros(T, fill(s[3], dims)...)
    for k in product(fill(1:s[2], dims)...)
        ret[(map(i -> seg(k[i], s[1], s[3]), 1:dims))...] = bsfunction(collect(k), bsdata...)
    end
    ret
end

bstoarray (generic function with 1 method)

In [26]:
using Tensors
seg(i::Int, of::Int, limit::Int) = (i*of <= limit)? (((i-1)*of+1):(i*of)): (((i-1)*of+1):limit)

issymetric{T <: AbstractFloat}(data::Array{T}, tool::Float64 = 1e-7) = map(i -> (maximum(abs(unfold(data, 1)-unfold(data, i))) < tool) || throw(DimensionMismatch("array is not symmetric")), 2:ndims(data))
segsizetest(len::Int, segments::Int) = ((len%segments) <= (len÷segments)) || throw(DimensionMismatch("last segment len $len-segments*(len÷segments)) > segment len $(len÷segments)"))
issquare{T <: AbstractFloat, S}(ar::(Union{NullableArrays.NullableArray{Array{T,S},S}, Array{T,S}})) = (maximum(size(ar)) == minimum(size(ar)) )

function creatnarray{T <: AbstractFloat}(data::Array{T}, rule::Function, segments::Int)
    (size(data,1)%segments == 0)? () : segments += 1
    dim = ndims(data)
    ret = NullableArray(Array{T, dim}, fill(segments, dim)...)
    for i in product(fill(1:segments, dim)...)
      if issorted(i)
        ret[i...] = rule(data, i...)::Array{T}
      end
    end
    ret
end


function covbb{T <: AbstractFloat}(data::Matrix{T}, segments::Int = 2, corrected::Bool = false)
    len = size(data,2)
    segsizetest(len, segments)
    (len%segments == 0)? () : segments += 1
    ret = NullableArray(Matrix{T}, segments, segments)
    for i = 1:segments, j = i:segments
        ret[i,j] = cov(data[:,seg(i, ceil(Int, len/segments), len)], data[:,seg(j, ceil(Int, len/segments), len)], corrected = corrected)
    end
    BoxStructure(ret)
end

covbb (generic function with 3 methods)

In [13]:
x = [2,3,3]
minimum(x .== x[1]) || throw(ArgumentError("underdiagonal block not null"))

LoadError: LoadError: ArgumentError: underdiagonal block not null
while loading In[13], in expression starting on line 2

In [5]:
a = randn(5,6)

5x6 Array{Float64,2}:
 -0.185857    0.515684  -1.77571   -0.0140652  -0.457079   -0.64563 
  0.0870488  -0.818033  -1.6023    -0.130673   -0.0775373   2.29581 
 -0.0823961  -1.20214   -3.34439    1.38618    -0.155304   -0.332532
 -0.401845    0.993415  -1.36642   -0.178862   -0.966669   -1.16284 
  0.726349    0.45505   -0.850471   0.214769   -0.80302     0.691576

In [22]:
for p in permutations([1,1])
    println(p)
end

[1,1]
[1,1]


In [27]:
ceil(Int, 10/5)

2

In [3]:
true * false

false

In [140]:
(1,2)

(1,2,3)